In [ ]:
from jupyter_plotly_dash import JupyterDash

import dash
import dash_leaflet as dl
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
import dash_table as dt
from dash.dependencies import Input, Output, State
import base64
import os
import numpy as np
import pandas as pd
from pymongo import MongoClient
from bson.json_util import dumps

#### FIX ME #####
# change animal_shelter and AnimalShelter to match your CRUD Python module file name and class name
from crud import CRUD

###########################
# Data Manipulation / Model
###########################
# FIX ME update with your username and password and CRUD Python module name

## Comment out pre-existing code to instantiate CRUD properly
#username = "aacuser"
#password = "changeme"
#shelter = AnimalShelter(username, password)


## Insert my CRUD instantiation data and constructor call
db = "AAC"
col = "animals"
usr = "aacuser"
passwd = "aacuser"
url = "localhost"
port = "54357"
shelter = CRUD(db, col, usr, passwd, url, port)

# class read method must support return of cursor object and accept projection json input
dataFilter = {}
df = pd.DataFrame.from_records(shelter.readAll(dataFilter))

#########################
# Dashboard Layout / View
#########################
app = JupyterDash('SimpleExample')

#FIX ME Add in Grazioso Salvare’s logo
image_filename = 'Grazioso Salvare Logo.png' # replace with your own image
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

#FIX ME Place the HTML image tag in the line below into the app.layout code according to your design
#FIX ME Also remember to include a unique identifier such as your name or date
#html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()))

app.layout = html.Div([
#    html.Div(id='hidden-div', style={'display':'none'}),
    html.Center(html.B(html.H1('SNHU CS-340 Dashboard'))),
    html.Hr(),
    html.Div(
        className='icon',
        style={ 'textAlign':'center' },
        children=[
            html.Img(
                src='data:image/png;base64,{}'.format(encoded_image.decode()),
                style={
                    'width':'10%'
    })]),

    html.H4('Developed by James Kraatz', style={ 'textAlign':'center' }),
    html.Hr(),
    html.Div(
        className='row',
        style={ 'display' : 'flex' },
        children=[
            
#FIXME Add in code for the interactive filtering options. For example, Radio buttons, drop down, checkboxes, etc.
            dcc.RadioItems( 
                id='filter-type',
                options=[
                    {'label':'Water Rescue', 'value': 'water'},
                    {'label':'Mountain or Wilderness Rescue', 'value': 'mountain'},
                    {'label':'Disaster Rescue or Individual Tracking', 'value': 'disaster'},
                    {'label':'Reset', 'value': 'reset'}
                ],
                value='reset',
                labelStyle={'display':'inline-block'}
    )]),
    html.Hr(),
    dt.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        data=df.to_dict('records'),
#FIXME: Set up the features for your interactive data table to make it user-friendly for your client
#If you completed the Module Six Assignment, you can copy in the code you created here 
        editable=False,
        filter_action="native",
        sort_action="native",
        sort_mode="multi",
        column_selectable=False,
        row_selectable="single",
        row_deletable=False,
        selected_columns= [],
        selected_rows= [],
        page_action="native",
        page_current=0,
        page_size=10,        
    ),
    html.Br(),
    html.Hr(),
#This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
        html.Div(
            id='graph-id',
            className='col s12 m6',

            ),
        html.Div(
            id='map-id',
            className='col s12 m6',
            )
        ])
])

#############################################
# Interaction Between Components / Controller
#############################################
@app.callback([Output('datatable-id','data'),
               Output('datatable-id','columns')],
              [Input('filter-type', 'value')])
def update_dashboard(filter_type):
### FIX ME Add code to filter interactive data table with MongoDB queries
    if   (filter_type=='water'):
        filter = {  "breed":{ "$in":["Labrador Retriever Mix", 
                            "Chesapeake Bay Retriever",
                            "Newfoundland"] },
            "sex_upon_outcome":"Intact Female",
            "age_upon_outcome_in_weeks": { "$gte":26, "$lte":156 }
        }
    elif (filter_type=='mountain'):
        filter = {
            "breed":{"$in":["German Shepherd",
                            "Alaskan Malamute",
                            "Old English Sheepdog",
                            "Siberian Husky",
                            "Rottweiler"]},
            "sex_upon_outcome":"Intact Male",
            "age_upon_outcome_in_weeks": { "$gte":26, "$lte":156 }
        }
    elif (filter_type=='disaster'):
        filter = {
            "breed" : {"$in":["Doberman Pinscher",
                            "German Shepherd",
                            "Golden Retriever",
                            "Bloodhound",
                            "Rottweiler"]},
            "sex_upon_outcome":"Intact Male",
            "age_upon_outcome_in_weeks": { "$gte":26, "$lte":300 }
        }
    else:   #(filter_type=='reset'):
        filter = {}
    
    if ( filter == {} ):
        df = pd.DataFrame.from_records(shelter.readAll(filter))
    else:
        df = pd.DataFrame(list(shelter.readAll(filter)))
        
    columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns]
    data=df.to_dict('records')

    return (data,columns)


@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]

@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_virtual_data")])
#    [Input('datatable-id', "data")])
def update_graphs(viewData):
    ###FIX ME ####
    # add code for chart of your choice (e.g. pie chart) #
    dff = df if viewData is None else pd.DataFrame(viewData)
    return [
        dcc.Graph(            
            figure = px.pie(dff, names=dff['breed'])
        )    
    ]

@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_viewport_data"),
    Input('datatable-id',"derived_viewport_selected_rows")])
def update_map(viewData,selectedRows):
#FIXME: Add in the code for your geolocation chart
#If you completed the Module Six Assignment, you can copy in the code you created here.

    viewDF = pd.DataFrame.from_dict(viewData)
    
    if(selectedRows == []):
        row = 0
    elif (selectedRows != []):
        row = selectedRows[0]
    
    dff=viewDF.iloc[row]

    # get data from selected row
    locLat=dff["location_lat"]
    locLong=dff["location_long"]
    breed=dff["breed"]
    name=dff["name"]
    
    # Austin TX is at [30.75,-97.48]
    return [
        dl.Map(style={'width': '1000px', 'height': '500px'}, center=[locLat,locLong], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            # Marker with tool tip and popup
            dl.Marker(position=[locLat,locLong], children=[
                dl.Tooltip(breed),
                dl.Popup([
                    html.P("Animal Name"),
                    html.P(name)
                ])
            ])
        ])
    ]

app